## Tutorial for CSVtoRDF

https://stackoverflow.com/questions/43524943/creating-rdf-file-using-csv-file-as-input

In [1]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
from rdflib.namespace import FOAF, XSD #most common namespaces
import urllib.parse #for parsing strings to URI's
import pandas as pd

In [5]:
url='https://raw.githubusercontent.com/KRontheWeb/csv2rdf-tutorial/master/example.csv'
df=pd.read_csv(url,sep=";",quotechar='"')

In [6]:
df

,Name,Address,Place,Country,Age,Hobby,Favourite Colour
0,John,Dam 52,Amsterdam,The Netherlands,32,Fishing,Blue
1,Jenny,Leidseplein 2,Amsterdam,The Netherlands,12,Dancing,Mauve
2,Jill,52W Street 5,Amsterdam,United States of America,28,Carpentry,Cyan
3,Jake,12E Street 98,Amsterdam,United States of America,42,Ballet,Purple


In [7]:
g = Graph()
ppl = Namespace('http://example.com/people/')
loc = Namespace('http://mylocations.org/addresses/')
schema = Namespace('http://schema.org/')

In [8]:
for index, row in df.iterrows():
    g.add((URIRef(ppl+row['Name']), RDF.type, FOAF.Person))
    g.add((URIRef(ppl+row['Name']), URIRef(schema+'name'), Literal(row['Name'], datatype=XSD.string) ))
    g.add((URIRef(ppl+row['Name']), FOAF.age, Literal(row['Age'], datatype=XSD.integer) ))
    g.add((URIRef(ppl+row['Name']), URIRef(schema+'address'), Literal(row['Address'], datatype=XSD.string) ))
    g.add((URIRef(loc+urllib.parse.quote(row['Address'])), URIRef(schema+'name'), Literal(row['Address'], datatype=XSD.string) ))

In [10]:
print(g.serialize(format='turtle'))
g.serialize('example.ttl',format='turtle')

@prefix ns1: <http://xmlns.com/foaf/0.1/> .
@prefix ns2: <http://schema.org/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.com/people/Jake> a ns1:Person ;
    ns2:address "12E Street 98"^^xsd:string ;
    ns2:name "Jake"^^xsd:string ;
    ns1:age 42 .

<http://example.com/people/Jenny> a ns1:Person ;
    ns2:address "Leidseplein 2"^^xsd:string ;
    ns2:name "Jenny"^^xsd:string ;
    ns1:age 12 .

<http://example.com/people/Jill> a ns1:Person ;
    ns2:address "52W Street 5"^^xsd:string ;
    ns2:name "Jill"^^xsd:string ;
    ns1:age 28 .

<http://example.com/people/John> a ns1:Person ;
    ns2:address "Dam 52"^^xsd:string ;
    ns2:name "John"^^xsd:string ;
    ns1:age 32 .

<http://mylocations.org/addresses/12E%20Street%2098> ns2:name "12E Street 98"^^xsd:string .

<http://mylocations.org/addresses/52W%20Street%205> ns2:name "52W Street 5"^^xsd:string .

<http://mylocations.org/addresses/Dam%2052> ns2:name "Dam 52"^^xsd:string .

<http://mylocations.org/addre

<Graph identifier=N42150f30038f4cdaa20c991a4360946a (<class 'rdflib.graph.Graph'>)>

In [1]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
import urllib.parse #for parsing strings to URI's
import pandas as pd
from pathlib import Path

main_path = Path().absolute().parent
data_path = main_path / 'data'

In [35]:
# full
df = pd.read_csv(data_path / 'AccountRDF.csv', encoding='utf-8')
ns_acc = Namespace('http://fsqa.com/acc#')
namespace_dict = {
    'acc': ns_acc, 'time': TIME, 'rdf': RDF, 'rdfs': RDFS, None: ''
}
g = Graph()
g.bind('rdf', RDF)
g.bind('time', TIME)
g.bind('acc', ns_acc, override=True)
for index, row in df.iterrows():
    s_ns, s = row['subject'].split(':')
    p_ns, p = row['predicate'].split(':')
    if len(row['object'].split(':')) < 2:
        o_ns, o = None, row['object']
    else:
        o_ns, o = row['object'].split(':')
    
    if o_ns is None:
        g.add( (URIRef(namespace_dict[s_ns]+s), URIRef(namespace_dict[p_ns]+p), Literal(o, datatype=XSD.string)) )
    else:
        g.add( (URIRef(namespace_dict[s_ns]+s), URIRef(namespace_dict[p_ns]+p), URIRef(namespace_dict[o_ns]+o)) )

In [36]:
g.serialize(data_path / 'AccountRDF.ttl', format='turtle')
g.serialize(data_path / 'AccountRDF.xml', format='xml', encoding='utf-8')

<Graph identifier=N35b60fcf28484699b962e16d48526170 (<class 'rdflib.graph.Graph'>)>

In [2]:
g = Graph()
g.load(data_path / 'AccountRDF.ttl', format='ttl')

<Graph identifier=N62d07776ddfd4224964baae40fa9727a (<class 'rdflib.graph.Graph'>)>

In [3]:
# show only BalanceSheet
query_string = """
SELECT DISTINCT ?acc
WHERE {
    ?acc acc:Account_Belonging acc:BalanceSheet .
}
"""
qres = g.query(query_string)
for x in qres:
    print(f'{x}')

(rdflib.term.URIRef('http://fsqa.com/acc#AdvancesCustomers'),)
(rdflib.term.URIRef('http://fsqa.com/acc#BondsIssued'),)
(rdflib.term.URIRef('http://fsqa.com/acc#CashAndCashEquivalents'),)
(rdflib.term.URIRef('http://fsqa.com/acc#IntangibleAssets'),)
(rdflib.term.URIRef('http://fsqa.com/acc#Inventories'),)
(rdflib.term.URIRef('http://fsqa.com/acc#LongTermBorrowings'),)
(rdflib.term.URIRef('http://fsqa.com/acc#PrepaidExpenses'),)
(rdflib.term.URIRef('http://fsqa.com/acc#PropertyPlantAndEquipment'),)
(rdflib.term.URIRef('http://fsqa.com/acc#ShorttermBorrowings'),)
(rdflib.term.URIRef('http://fsqa.com/acc#TotalEquity'),)
(rdflib.term.URIRef('http://fsqa.com/acc#TradeAndOtherCurrentPayables'),)
(rdflib.term.URIRef('http://fsqa.com/acc#TradeAndOtherCurrentReceivables'),)
(rdflib.term.URIRef('http://fsqa.com/acc#AssetsAbstract'),)
(rdflib.term.URIRef('http://fsqa.com/acc#LiabilitiesAbstract'),)
(rdflib.term.URIRef('http://fsqa.com/acc#LiabilitiesAndEquities'),)
(rdflib.term.URIRef('http://fsq

In [39]:
# relationship BalanceSheet
query_statement = """
CONSTRUCT {
    ?s acc:partOf acc:CurrentAsset ;
}
WHERE {
    ?s acc:Account_Belonging acc:BalanceSheet .
    ?o acc:Account_Belonging acc:BalanceSheet .
    FILTER NOT EXISTS {
        ?s acc:partOf ?o
    }
}
"""
qres = g.query(query_statement)
len(list(qres))

ParseException: Expected {SelectQuery | ConstructQuery | DescribeQuery | AskQuery}, found 'F'  (at char 160), (line:8, col:5)

In [32]:
for subj, pred, obj in qres:
    print(subj.split('#')[1], pred.split('#')[1], obj.split('#')[1])

LongTermBorrowings partOf Inventories
AssetsAbstract partOf TradeAndOtherCurrentPayables
Inventories partOf AssetsAbstract
LongTermBorrowings partOf LongTermBorrowings
PropertyPlantAndEquipment partOf AdvancesCustomers
ShorttermBorrowings partOf AdvancesCustomers
PrepaidExpenses partOf LongTermBorrowings
Inventories partOf TradeAndOtherCurrentPayables
TradeAndOtherCurrentReceivables partOf TotalEquity
TotalEquity partOf PrepaidExpenses
LongTermBorrowings partOf TradeAndOtherCurrentPayables
CashAndCashEquivalents partOf CurrentLiabilities
CashAndCashEquivalents partOf CurrentAssets
Inventories partOf TotalEquity
TradeAndOtherCurrentReceivables partOf PrepaidExpenses
AdvancesCustomers partOf PropertyPlantAndEquipment
AdvancesCustomers partOf ShorttermBorrowings
BondsIssued partOf AdvancesCustomers
BondsIssued partOf TradeAndOtherCurrentReceivables
Inventories partOf LiabilitiesAbstract
Inventories partOf NoncurrentAssets
TradeAndOtherCurrentPayables partOf PropertyPlantAndEquipment
Trade

In [17]:
from streamlit_agraph import TripleStore, agraph, Config, GraphAlgos


In [35]:
import networkx as nx
import matplotlib.pyplot as plt
from pyvis.network import Network

In [37]:
net = Network(height="600px", width="100%", font_color="black",heading='Game of Thrones Graph')

<Graph identifier=N62d07776ddfd4224964baae40fa9727a (<class 'rdflib.graph.Graph'>)>

In [ ]:
for 